In [66]:
import requests
import zipfile
import time
import os
import unicodedata
from urllib.parse import urljoin
from io import BytesIO
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [70]:
sslverify = True  # nastavit na False u serveru, ktere hlasi chybu ssl
urlRoot = "https://volby.cz/opendata/"
opendatafile = "opendata.htm"
datadir = "data"

In [56]:
opendataUrl = urljoin(urlRoot,opendatafile)
response = requests.get(opendataUrl)

https://volby.cz/opendata/opendata.htm


In [4]:
def getUrlsVolby(soup):
    volbyList = []
    for volby in soup.table.find_all("tr"):
        volbyData = volby.find("td").text
        volbyData = unicodedata.normalize('NFKD',volbyData)
        #print(volbyData)
        #print(volba)
        volbyUrls = volby.find_all("a")
        for i in volbyUrls:
            volba = []
            volba.append(volbyData)
            url = i.attrs["href"]
            # print(url)
            volbyId, volbyFileurl = url.split("/")
            volba.append(volbyId)
            volba.append(volbyFileurl)
            volbyList.append(volba)
            # print(volba)
            # print()
    return(volbyList)

In [5]:
if response.status_code == 200:
    HomeSoup = BeautifulSoup(response.content, 'html.parser')
    volbyUrls = getUrlsVolby(HomeSoup)
else:
    print("chyba zpracování", opendataUrl)

In [7]:
def zpracujOdata(url):
    odkazy = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for i in soup.find_all("li"):
        for a in i.find_all("a"):
            odkaz = a.attrs["href"]
            if "http" not in odkaz:
                if "_opendata.htm" in odkaz:
                    odkaz = odkaz.replace("..","/opendata")
                    odkaz = urljoin("https://volby.cz", odkaz)
                    odkazy.append(odkaz)
    return(odkazy)


volbaOpendataHome = []
for i in volbyUrls:
    volbaUrl = urlRoot + i[1] + "/" + i[2]
    time.sleep(0.3)
    if "odata" in volbaUrl or "_seznam" in volbaUrl:
        odkazy = zpracujOdata(volbaUrl)
        for odkaz in odkazy:
            volbaOpendataHome.append(odkaz)
    else:
        volbaOpendataHome.append(volbaUrl)

In [ ]:
sorted(volbaOpendataHome, reverse=True)

In [57]:
def getVolbySoup(home, sleeptime):
    responses = {}
    for i in home:
        volby = i.split("/")[4]
        responses[volby] = requests.get(i)
        time.sleep(sleeptime)
    return (responses)


volbySoupsDict = getVolbySoup(volbaOpendataHome, 2)

In [ ]:
volbySoupsDict

In [97]:
def getDavky(davkydir, url):
    posledni_davka = requests.get(url)
    root = ET.fromstring(posledni_davka.text)
    davka = root[0].attrib
    davkyLimit = int(davka.get("PORADI_DAVKY")) + 1
    if not os.path.exists(davkydir):
        os.mkdir(davkydir)
    for i in range(1, davkyLimit):
        response = requests.get(url, verify=sslverify)
        time.sleep(1)
        print("davka", i, response.status_code, "\r")
        if i < 10:
            davkaurl = os.path.join(davkydir, "davka_0"+str(i)+".xml")
        else:
            davkaurl = os.path.join(davkydir, "davka_"+str(i)+".xml")
        with open(davkaurl, "wb") as file:
            file.write(response.content)


def getExtractZipDavky(volbadir, volbyDict):
    for volba, response in volbyDict.items():
        volba = os.path.join(volbadir, volba)
        if not os.path.exists(volba):
            os.mkdir(volba)
        url = response.url
        soup = BeautifulSoup(response.content, 'html.parser')
        for a in soup.find_all('a'):
            odkaz = a['href']
            if '.zip' in odkaz:
                continue  # docasne zablokovat download zip
                fileUrl = urljoin(url, odkaz)
                dataType = fileUrl.split('/')[4]
                path = os.path.join(volba,dataType)
                r = requests.get(fileUrl)
                if r.status_code == 200:
                    z = zipfile.ZipFile(BytesIO(r.content))
                    z.extractall(path)
                time.sleep(1)
            elif 'davka=' in odkaz:
                print("odkaz", odkaz)
                fileUrl = urljoin(url, odkaz)
                print("fileUrl", fileUrl)
                dataType = fileUrl.split('/')[4]
                path = os.path.join(volba,dataType)
                if 'se' in volba or 'prez' in volba:
                    for i in range(1,3):
                        if 'kola=' in fileUrl:
                            fileUrl = fileUrl.replace("kola=","kola="+str(i))
                        else:
                            fileUrl = fileUrl+"&kola="+str(i)
                        getDavky(path, fileUrl)
                else:
                    print(fileUrl)
                    getDavky(path, fileUrl)
            else:
                continue
        time.sleep(1)


if not os.path.exists(datadir):
    os.mkdir(datadir)

getExtractZipDavky(datadir, volbySoupsDict)

odkaz /pls/prez2018/vysledky_okrsky?davka=
fileUrl https://www.volby.cz/pls/prez2018/vysledky_okrsky?davka=


ParseError: syntax error: line 1, column 49 (<string>)

In [ ]:
def getDavky(dataDir, volbyDict):
    
    def get_posledni_davka(urlDavky):
    posledni_davka = requests.get(url_davky,verify=sslverify)
    root = ET.fromstring(posledni_davka.text)
    #for child in root:
    #    print (child.tag,child.attrib)
    davka = root[0].attrib
    davky_pocet = int(davka.get("PORADI_DAVKY")) + 1
    return(davky_pocet)

davky_pocet_kolo1 = get_posledni_davka(url_davky+"&kolo=1")
davky_pocet_kolo2 = get_posledni_davka(url_davky+"&kolo=2")
print(davky_pocet_kolo1,davky_pocet_kolo2)

def get_davky(kolo,davky_pocet):
    davkydir = os.path.join(volby,"davky"+kolo)
    dir_exists = os.path.exists(davkydir)
    if not dir_exists:
        os.mkdir(davkydir)
    for i in range(1,davky_pocet):
        url=url_davky+str(i)+"&kolo="+kolo
        response=requests.get(url,verify=sslverify)
        time.sleep(1)
        print("davka",i,response.status_code,"\r")
        if i<10:
            davkaurl=os.path.join(davkydir,"davka_0"+str(i)+".xml")                                
        else:
            davkaurl=os.path.join(davkydir,"davka_"+str(i)+".xml")
        with open(davkaurl,"wb") as file:
            file.write(response.content)
get_davky("1",davky_pocet_kolo1)
get_davky("2",davky_pocet_kolo2)    

In [ ]:
volby = "prez2018"
ciselniky = "PREZ2018ciselniky20171215_csv.zip"
registry = "PREZ2018reg20180127_csv.zip"
okrskovaData = "PREZ2018data20180127_csv_kolo2.zip"
urlOpendata = urlRoot+volby+"/"+volby+"_"+opendatafileHome
urlDavky = urlRootDavky+volby+"/"+davky
urlCiselniky = urlRoot+volby+"/"+ciselniky
urlRegistry = urlRoot+volby+"/"+registry
urlOkrskovaData = urlRoot+volby+"/"+okrskovaData
davky = True
kola = True

In [ ]:
if not os.path.exists(volby):
    os.mkdir(volby)

In [ ]:
def get_posledni_davka(urlDavky):
    posledni_davka = requests.get(url_davky,verify=sslverify)
    root = ET.fromstring(posledni_davka.text)
    #for child in root:
    #    print (child.tag,child.attrib)
    davka = root[0].attrib
    davky_pocet = int(davka.get("PORADI_DAVKY")) + 1
    return(davky_pocet)

In [ ]:
davky_pocet_kolo1 = get_posledni_davka(url_davky+"&kolo=1")
davky_pocet_kolo2 = get_posledni_davka(url_davky+"&kolo=2")
print(davky_pocet_kolo1,davky_pocet_kolo2)

In [ ]:
def get_data_voleb (volby,davky,kola,ciselniky,registry,okrskovaData)

In [ ]:
def get_davky(kolo,davky_pocet):
    davkydir = os.path.join(volby,"davky"+kolo)
    dir_exists = os.path.exists(davkydir)
    if not dir_exists:
        os.mkdir(davkydir)
    for i in range(1,davky_pocet):
        url=url_davky+str(i)+"&kolo="+kolo
        response=requests.get(url,verify=sslverify)
        time.sleep(1)
        print("davka",i,response.status_code,"\r")
        if i<10:
            davkaurl=os.path.join(davkydir,"davka_0"+str(i)+".xml")
        else:
            davkaurl=os.path.join(davkydir,"davka_"+str(i)+".xml")
        with open(davkaurl,"wb") as file:
            file.write(response.content)
get_davky("1",davky_pocet_kolo1)
get_davky("2",davky_pocet_kolo2) 

In [ ]:
def get_cisregokr():
    for url in [url_ciselniky,url_registry,url_okrskova_data]:
        response = requests.get(url,verify=sslverify)
        zipfile.ZipFile(BytesIO(response.content)).extractall(path=volby)
get_cisregokr()

In [ ]:
response = requests.get(url_ciselniky)

In [ ]:
response.url.split("/")[-1]